In [1]:
import json

# Employee Class
class Employee:
    def __init__(self, employee_id, name, department, position, salary):
        self.employee_id = employee_id
        self.name = name
        self.department = department
        self.position = position
        self.salary = salary
        self.promotion_history = []
        self.performance_reviews = []

    def add_review(self, review):
        self.performance_reviews.append(review)

    def promote(self, new_position, new_salary):
        self.promotion_history.append(self.position)
        self.position = new_position
        self.salary = new_salary

# Department Class
class Department:
    def __init__(self, department_id, name):
        self.department_id = department_id
        self.name = name
        self.employees = []

    def add_employee(self, employee):
        self.employees.append(employee)

    def get_performance_summary(self):
        scores = [review.review_score for emp in self.employees for review in emp.performance_reviews]
        avg_score = sum(scores) / len(scores) if scores else 0
        return {
            "Total Employees": len(self.employees),
            "Average Performance": avg_score
        }

# PerformanceReview Class
class PerformanceReview:
    def __init__(self, review_id, employee_id, date, review_score, comments):
        self.review_id = review_id
        self.employee_id = employee_id
        self.date = date
        self.review_score = review_score
        self.comments = comments

# File Handling (Optional)
def save_to_file(data, filename="data.json"):
    with open(filename, 'w') as file:
        json.dump(data, file)

def load_from_file(filename="data.json"):
    try:
        with open(filename, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return {}


In [2]:
emp1 = Employee(1, "John Doe", "HR", "Manager", 60000)


In [3]:
dept1 = Department(1, "HR")
dept1.add_employee(emp1)


In [4]:
review = PerformanceReview(1, 1, "2024-12-01", 4.5, "Excellent work")
emp1.add_review(review)


In [5]:
print(dept1.get_performance_summary())


{'Total Employees': 1, 'Average Performance': 4.5}


In [6]:
import json
import tkinter as tk
from tkinter import ttk, messagebox

# Employee Class
class Employee:
    def __init__(self, employee_id, name, department, position, salary):
        self.employee_id = employee_id
        self.name = name
        self.department = department
        self.position = position
        self.salary = salary
        self.promotion_history = []
        self.performance_reviews = []

    def add_review(self, review):
        self.performance_reviews.append(review)

    def promote(self, new_position, new_salary):
        self.promotion_history.append({"position": self.position, "salary": self.salary})
        self.position = new_position
        self.salary = new_salary

# Department Class
class Department:
    def __init__(self, department_id, name):
        self.department_id = department_id
        self.name = name
        self.employees = []

    def add_employee(self, employee):
        self.employees.append(employee)

    def get_performance_summary(self):
        scores = [review.review_score for emp in self.employees for review in emp.performance_reviews]
        avg_score = sum(scores) / len(scores) if scores else 0
        return {
            "Department Name": self.name,
            "Total Employees": len(self.employees),
            "Average Performance": avg_score
        }

# PerformanceReview Class
class PerformanceReview:
    def __init__(self, review_id, employee_id, date, review_score, comments):
        self.review_id = review_id
        self.employee_id = employee_id
        self.date = date
        self.review_score = review_score
        self.comments = comments

# Data Persistence
class DataHandler:
    @staticmethod
    def save_to_file(data, filename="data.json"):
        with open(filename, 'w') as file:
            json.dump(data, file, default=lambda o: o.__dict__, indent=4)

    @staticmethod
    def load_from_file(filename="data.json"):
        try:
            with open(filename, 'r') as file:
                return json.load(file)
        except FileNotFoundError:
            return {}

# Main System Class
class EmployeeManagementSystem:
    def __init__(self):
        self.employees = {}
        self.departments = {}

    def add_employee(self, employee):
        self.employees[employee.employee_id] = employee
        if employee.department not in self.departments:
            print(f"Error: Department {employee.department} does not exist.")
            return
        self.departments[employee.department].add_employee(employee)

    def add_department(self, department):
        self.departments[department.name] = department

    def search_employee(self, **kwargs):
        for emp in self.employees.values():
            if all(getattr(emp, key) == value for key, value in kwargs.items()):
                return emp
        return None

    def promote_employee(self, employee_id, new_position, new_salary):
        employee = self.employees.get(employee_id)
        if not employee:
            print("Error: Employee not found.")
            return
        employee.promote(new_position, new_salary)

    def add_performance_review(self, employee_id, review):
        employee = self.employees.get(employee_id)
        if not employee:
            print("Error: Employee not found.")
            return
        employee.add_review(review)

    def generate_department_report(self, department_name):
        department = self.departments.get(department_name)
        if not department:
            print("Error: Department not found.")
            return
        return department.get_performance_summary()

    def save_data(self):
        data = {
            "employees": self.employees,
            "departments": self.departments
        }
        DataHandler.save_to_file(data)

    def load_data(self):
        data = DataHandler.load_from_file()
        self.employees = data.get("employees", {})
        self.departments = data.get("departments", {})

# GUI Application
class EmployeeManagementApp:
    def __init__(self, root, system):
        self.system = system
        self.root = root
        self.root.title("Employee Management System")
        
        # Tabs
        self.tab_control = ttk.Notebook(root)
        self.tab_employee = ttk.Frame(self.tab_control)
        self.tab_department = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab_employee, text="Employees")
        self.tab_control.add(self.tab_department, text="Departments")
        self.tab_control.pack(expand=1, fill="both")

        # Employee Tab
        self.create_employee_tab()
        self.create_department_tab()

    def create_employee_tab(self):
        # Labels and Entries
        ttk.Label(self.tab_employee, text="Employee ID:").grid(row=0, column=0, padx=10, pady=5)
        self.emp_id = ttk.Entry(self.tab_employee)
        self.emp_id.grid(row=0, column=1, padx=10, pady=5)

        ttk.Label(self.tab_employee, text="Name:").grid(row=1, column=0, padx=10, pady=5)
        self.emp_name = ttk.Entry(self.tab_employee)
        self.emp_name.grid(row=1, column=1, padx=10, pady=5)

        ttk.Label(self.tab_employee, text="Department:").grid(row=2, column=0, padx=10, pady=5)
        self.emp_department = ttk.Entry(self.tab_employee)
        self.emp_department.grid(row=2, column=1, padx=10, pady=5)

        ttk.Label(self.tab_employee, text="Position:").grid(row=3, column=0, padx=10, pady=5)
        self.emp_position = ttk.Entry(self.tab_employee)
        self.emp_position.grid(row=3, column=1, padx=10, pady=5)

        ttk.Label(self.tab_employee, text="Salary:").grid(row=4, column=0, padx=10, pady=5)
        self.emp_salary = ttk.Entry(self.tab_employee)
        self.emp_salary.grid(row=4, column=1, padx=10, pady=5)

        # Buttons
        ttk.Button(self.tab_employee, text="Add Employee", command=self.add_employee).grid(row=5, column=0, columnspan=2, pady=10)
        ttk.Button(self.tab_employee, text="View Employee", command=self.view_employee).grid(row=6, column=0, columnspan=2, pady=10)

    def create_department_tab(self):
        ttk.Label(self.tab_department, text="Department ID:").grid(row=0, column=0, padx=10, pady=5)
        self.dept_id = ttk.Entry(self.tab_department)
        self.dept_id.grid(row=0, column=1, padx=10, pady=5)

        ttk.Label(self.tab_department, text="Name:").grid(row=1, column=0, padx=10, pady=5)
        self.dept_name = ttk.Entry(self.tab_department)
        self.dept_name.grid(row=1, column=1, padx=10, pady=5)

        ttk.Button(self.tab_department, text="Add Department", command=self.add_department).grid(row=2, column=0, columnspan=2, pady=10)
        ttk.Button(self.tab_department, text="Generate Report", command=self.generate_report).grid(row=3, column=0, columnspan=2, pady=10)

    def add_employee(self):
        try:
            emp_id = int(self.emp_id.get())
            name = self.emp_name.get()
            department = self.emp_department.get()
            position = self.emp_position.get()
            salary = float(self.emp_salary.get())

            employee = Employee(emp_id, name, department, position, salary)
            self.system.add_employee(employee)
            messagebox.showinfo("Success", "Employee added successfully!")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def view_employee(self):
        try:
            emp_id = int(self.emp_id.get())
            employee = self.system.employees.get(emp_id)
            if not employee:
                messagebox.showinfo("Not Found", "Employee not found.")
                return

            details = f"ID: {employee.employee_id}\nName: {employee.name}\nDepartment: {employee.department}\nPosition: {employee.position}\nSalary: {employee.salary}"
            messagebox.showinfo("Employee Details", details)
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def add_department(self):
        try:
            dept_id = int(self.dept_id.get())
            name = self.dept_name.get()
            department = Department(dept_id, name)
            self.system.add_department(department)
            messagebox.showinfo("Success", "Department added successfully!")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def generate_report(self):
        try:
            name = self.dept_name.get()
            report = self.system.generate_department_report(name)
            if not report:
                messagebox.showinfo("Not Found", "Department not found.")
                return

            details = f"Department: {report['Department Name']}\nTotal Employees: {report['Total Employees']}\nAverage Performance: {report['Average Performance']}"
            messagebox.showinfo("Department Report", details)
        except Exception as e:
            messagebox.showerror("Error", str(e))

# Example Usage
if __name__ == "__main__":
    system = EmployeeManagementSystem()
    root = tk.Tk()
    app = EmployeeManagementApp(root, system)
    root.mainloop()
